# Data Preparation

- This file deals with preparing data for analysizing various graphs from `pysal_narsc2022 Tutorial`.

- Data used in this file can be found in `UNL-Hawkins_Research_Group>Documents>EV-Chicken and Egg Project>Data>TransportExperian Registrations>Combined files for years`

- GIS data can be found in `UNL-Hawkins_Research_Group>Documents>EV-Chicken and Egg Project>Data>GIS`


---

In [1]:
import pandas as pd
import geopandas as gpd
import dask.dataframe as dd

path_to_data = '/Users/sagunkarki/Desktop/UNL_Hawkins_Research_Group/geo_viz/graphs/data'

These are the vehicle groups we are focusing on

In [2]:
fuel_types = {
    'ICE': ['Gasoline', 'Diesel', 'Biodiesel', 'Flex Fuel', 'Flex Fuel~Gasoline', 'LPG~Gasoline', 'CNG', 'Natural Gas', 'CNG~Flex~Gasoline', 'CNG~Gasoline', 'CNG~LPG~Gasoline', 'Diesel~Gasoline', 'E85~Flex~Gasoline', 'Ethanol', 'Flex', 'Flex~Gasoline', 'Flex~Gasoline/E85/CNG/LPG', 'GFP', 'Gasoline Hybrid', 'Gasoline Hybrid~Gasoline', 'Gasoline/LPG', 'Gasoline~Natural Gas', 'Hydrogen Fuel Cell', 'LPG', 'Methanol', 'Natural Gas/Propane', 'Propane'],
    'BEV': ['Electric', ],
    'PHEV': ['Plug-in Gas/Electric Hybrid',  'Plug-In Hybrid', 'Electric with Gas Generator',],
    'HEV': ['Electric~Gasoline Hybrid', 'Gas/Electric Hybrid', ],
    'nan': ['nan']
}

fuel_types

# for each of the values make a new dict mapping to keys
fuel_types_rev = {j: i for i in fuel_types.keys() for j in fuel_types[i]}


##### Load gp data as it has geometry data


We have created state_county to join gp with county data so that we donot lose geometry data for counties if we only join it by state and county code.

In [3]:
gp = gpd.read_file(f"{path_to_data}/GIS/cb_2018_us_county_5m/cb_2018_us_county_5m.shp")

gpx = gp[['STATEFP', 'COUNTYFP', 'geometry', 'GEOID']].copy()
gpx[['STATEFP', 'COUNTYFP']] = gpx[['STATEFP', 'COUNTYFP']].astype(int)

gpx['state_county'] = gpx.apply(lambda row: f"S{row['STATEFP']}_C{row['COUNTYFP']}", axis=1)
gpx = gpx.rename(columns={
'GEOID': 'geoid',
})

In [4]:
gpx.head()

,STATEFP,COUNTYFP,geometry,geoid,state_county
0,39,71,"POLYGON ((-83.86976 39.05553, -83.86568 39.247...",39071,S39_C71
1,6,3,"POLYGON ((-120.07248 38.50987, -120.07239 38.7...",6003,S6_C3
2,12,33,"POLYGON ((-87.62999 30.87766, -87.62946 30.880...",12033,S12_C33
3,17,101,"POLYGON ((-87.91028 38.57493, -87.90811 38.850...",17101,S17_C101
4,28,153,"POLYGON ((-88.94317 31.78421, -88.94335 31.824...",28153,S28_C153


Some basic tested data cleaning for county data

In [5]:
def load_county_data(
        year: int,
        gpx: gpd.GeoDataFrame = gpx,
):
    df = pd.read_parquet(f"{path_to_data}/COUNTY_{year}.parquet", columns=['State Code', 'County Code',  'Vehicle Fuel Type', 'Vehicle Count'])

    #Combine vehicle fuel types into categories
    df['Vehicle Fuel Type'] = df['Vehicle Fuel Type'].map(fuel_types_rev)

    df_grouped = df.groupby(['State Code', 'County Code', 'Vehicle Fuel Type'])['Vehicle Count'].sum().reset_index()

    df_grouped['state_county'] = df_grouped.apply(lambda row: f"S{row['State Code']}_C{row['County Code']}", axis=1)

    # We do not have geo data for county code 0
    df_grouped = df_grouped[df_grouped['County Code'] != 0]

    df_grouped['year'] = year

        # Merge on left as gpx has polygons for all counties
    return gpx.merge(df_grouped, on='state_county', how='left')[['STATEFP', 'COUNTYFP', 'geometry', 'Vehicle Fuel Type', 'Vehicle Count', 'year', "geoid"]]

### load data

In [6]:
# currently we have these years data
years = [2021, 2020, 2018, 2016, 2014, 2012]

# load data for all years and merge them
df = pd.concat([load_county_data(year, gpx) for year in years])

In [7]:
df

,STATEFP,COUNTYFP,geometry,Vehicle Fuel Type,Vehicle Count,year,geoid
0,39,71,"POLYGON ((-83.86976 39.05553, -83.86568 39.247...",BEV,15.0,2021.0,39071
1,39,71,"POLYGON ((-83.86976 39.05553, -83.86568 39.247...",HEV,322.0,2021.0,39071
2,39,71,"POLYGON ((-83.86976 39.05553, -83.86568 39.247...",ICE,41501.0,2021.0,39071
3,39,71,"POLYGON ((-83.86976 39.05553, -83.86568 39.247...",PHEV,17.0,2021.0,39071
4,39,71,"POLYGON ((-83.86976 39.05553, -83.86568 39.247...",nan,404.0,2021.0,39071
...,...,...,...,...,...,...,...
4112,12,1,"POLYGON ((-82.65855 29.83014, -82.65149 29.825...",ICE,11326.0,2012.0,12001
4113,48,247,"POLYGON ((-98.95467 27.26940, -98.79809 27.268...",ICE,964.0,2012.0,48247
4114,29,99,"POLYGON ((-90.76280 38.29555, -90.75560 38.330...",HEV,2.0,2012.0,29099
4115,29,99,"POLYGON ((-90.76280 38.29555, -90.75560 38.330...",ICE,18840.0,2012.0,29099


In [8]:
dfs = df.copy()
dfs = dfs.pivot_table(index=['STATEFP', 'COUNTYFP','geometry', 'geoid', 'year'], columns='Vehicle Fuel Type', values='Vehicle Count', aggfunc='sum').fillna(0).astype(int).reset_index()

In [9]:
# adding total of all fuel types
cols = fuel_types.keys()
dfs['total_ev'] = dfs[cols].sum(axis=1)
dfs

Vehicle Fuel Type,STATEFP,COUNTYFP,geometry,geoid,year,BEV,HEV,ICE,PHEV,nan,total_ev
0,1,1,"POLYGON ((-86.91759 32.66417, -86.81657 32.660...",1001,2012.0,0,0,4919,0,0,4919
1,1,1,"POLYGON ((-86.91759 32.66417, -86.81657 32.660...",1001,2014.0,1,15,5931,0,0,5947
2,1,1,"POLYGON ((-86.91759 32.66417, -86.81657 32.660...",1001,2016.0,5,6,7226,7,0,7244
3,1,1,"POLYGON ((-86.91759 32.66417, -86.81657 32.660...",1001,2018.0,13,8,8311,11,0,8343
4,1,1,"POLYGON ((-86.91759 32.66417, -86.81657 32.660...",1001,2020.0,23,10,9315,18,0,9366
...,...,...,...,...,...,...,...,...,...,...,...
18839,56,45,"POLYGON ((-105.08078 43.96622, -105.07928 44.1...",56045,2014.0,0,4,2134,0,0,2138
18840,56,45,"POLYGON ((-105.08078 43.96622, -105.07928 44.1...",56045,2016.0,0,2,2422,0,0,2424
18841,56,45,"POLYGON ((-105.08078 43.96622, -105.07928 44.1...",56045,2018.0,0,4,2504,1,0,2509
18842,56,45,"POLYGON ((-105.08078 43.96622, -105.07928 44.1...",56045,2020.0,1,2,2519,0,0,2522


### Export it to gpkg as it preserves geometry data

In [10]:
dfs['year'] = dfs['year'].astype(int)
dfs = gpd.GeoDataFrame(dfs, geometry='geometry')

dfs.dtypes

Vehicle Fuel Type
STATEFP        int64
COUNTYFP       int64
geometry    geometry
geoid         object
year           int64
BEV            int64
HEV            int64
ICE            int64
PHEV           int64
nan            int64
total_ev       int64
dtype: object

In [11]:
dfs.to_file(f"county_ev_data.gpkg")

---

In [12]:
import pandas as pd
year = 2021
df = dd.read_parquet(f"{path_to_data}/COUNTY_{year}.parquet")

In [13]:
df

,State Code,State Abbr,County Code,County Name,Vehicle Category,Vehicle Segmentation,Vehicle Manufacturer,Vehicle Make,Vehicle Model,Vehicle Model Year,Vehicle Series,Vehicle Body Style,Vehicle Door Count,Vehicle Drivewheel,Vehicle Fuel Type,Vehicle Engine Liters,Vehicle Engine Cylinders,Vehicle Performance Option,Vehicle Fuel Delivery,Vehicle Wheelbase,Vehicle GVW Class,Vehicle Weight,Vehicle Transmission Type,Vehicle Min HP,Vehicle Max HP,Vehicle Min kW,Vehicle Max kW,Vehicle Import Domestic HQ,Vehicle Import Domestic Sales,Fleet Flag,Ind/Org Flag,Vehicle Count
npartitions=1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,uint8,object,uint16,object,object,object,object,object,object,uint16,object,object,object,object,object,object,object,object,object,float64,object,float64,object,uint16,uint16,uint16,uint16,object,object,object,object,uint16
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [14]:
df.columns

Index(['State Code', 'State Abbr', 'County Code', 'County Name',
       'Vehicle Category', 'Vehicle Segmentation', 'Vehicle Manufacturer',
       'Vehicle Make', 'Vehicle Model', 'Vehicle Model Year', 'Vehicle Series',
       'Vehicle Body Style', 'Vehicle Door Count', 'Vehicle Drivewheel',
       'Vehicle Fuel Type', 'Vehicle Engine Liters',
       'Vehicle Engine Cylinders', 'Vehicle Performance Option',
       'Vehicle Fuel Delivery', 'Vehicle Wheelbase', 'Vehicle GVW Class',
       'Vehicle Weight', 'Vehicle Transmission Type', 'Vehicle Min HP',
       'Vehicle Max HP', 'Vehicle Min kW', 'Vehicle Max kW',
       'Vehicle Import Domestic HQ', 'Vehicle Import Domestic Sales',
       'Fleet Flag', 'Ind/Org Flag', 'Vehicle Count'],
      dtype='object')

In [15]:
df['Vehicle Category'].unique()

Dask Series Structure:
npartitions=1
    object
       ...
Name: Vehicle Category, dtype: object
Dask Name: unique-agg, 4 graph layers